# Datasets imported
DPPS

PHYSICAL

Z


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Read the CSV files
train_X = pd.read_csv('train_X.csv')
dpps = pd.read_csv('DPPS.csv', skiprows=2)
dpps= dpps.drop('AA_3', axis =1)
dpps_used= dpps[["AA_1","D7", "D8"]]
phys = pd.read_csv("Physical.csv", skiprows=2)
phys= phys.drop('AA_3', axis =1)
zs = pd.read_csv("Z-scale.csv", skiprows=2)
zs= zs.drop('AA_3', axis =1)
zs= zs.drop('Z(3)', axis =1)
vhse = pd.read_csv("VHSE-scale.csv", skiprows=2)
vhse= vhse.drop('AA_3', axis =1)
vhse_used= vhse[["AA_1","VHSE1", "VHSE2"]]
mswhim = pd.read_csv("MS-WHIM.csv", skiprows=2)
mswhim_used = mswhim[["AA_1", "2"]]
st = pd.read_csv("ST-scale.csv", skiprows=2)
st_used = st[["AA_1", "ST3", "ST4", "ST5"]]

# Clean up the DPPS dataframe
dpps_used.columns = [ 'AA_1'] + [f'D{i}' for i in range(7, 9)]
dpps_dict = dpps_used.set_index('AA_1').to_dict('index')
phys_used= phys[["AA_1","Vol", "Hydro"]]
phys_dict = phys_used.set_index('AA_1').to_dict('index')
zs_dict = zs.set_index('AA_1').to_dict('index')
vhse_dict = vhse_used.set_index('AA_1').to_dict('index')
mswhim_dict = mswhim_used.set_index('AA_1').to_dict('index')
st_dict = st_used.set_index('AA_1').to_dict('index')
def seq_to_dpps(seq):
    result = {}
    for i, aa in enumerate(seq):
        if aa in dpps_dict:
            for j in range(7, 9):
                result[f'pos_{i+1}_D{j}'] = dpps_dict[aa][f'D{j}']
        if aa in phys_dict:
            result[f'pos_{i+1}_Phys(Hydro)'] = phys_dict[aa]["Hydro"]
        if aa in zs_dict:
            result[f'pos_{i+1}_Z(1)'] = zs_dict[aa]["Z(1)"]
            result[f'pos_{i+1}_Z(2)'] = zs_dict[aa]["Z(2)"]
        if aa in vhse_dict:
            result[f'pos_{i+1}_VHSE1'] = vhse_dict[aa]["VHSE1"]
            result[f'pos_{i+1}_VHSE2'] = vhse_dict[aa]["VHSE2"]
        if aa in mswhim_dict:
            result[f'pos_{i+1}_MSWHIM1'] = mswhim_dict[aa]["2"]
        if aa in st_dict:
            for j in range(3, 6):
                result[f'pos_{i+1}_ST{j}'] = st_dict[aa][f'ST{j}']
        
    return result

# Apply the function to each sequence
dpps_features = train_X['ConstructedAASeq_cln'].apply(seq_to_dpps)

# Convert the result to a DataFrame

# Use 'Id' as the target variable


In [2]:
y = pd.read_csv("train_y.csv")
Y = y.Brightness_Class

In [3]:
X = pd.DataFrame(dpps_features.tolist())

# Split the data without stratification
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42, stratify=Y)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Training set size:", len(X_train))
print("Test set size:", len(X_test))

Training set size: 27926
Test set size: 3103


In [4]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, y_train)

# Evaluate the model
train_score = model.score(X_train_scaled, y_train)
test_score = model.score(X_test_scaled, y_test)

print(f"Train accuracy: {train_score:.4f}")
print(f"Test accuracy: {test_score:.4f}")

Train accuracy: 0.9218
Test accuracy: 0.8959


C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [5]:
from sklearn.metrics import f1_score, precision_score
f1_score(y_test, model.predict(X_test_scaled))

0.8687525396180414

In [6]:
precision_score(y_test, model.predict(X_test_scaled))

0.8531524341580208

In [7]:
test_x = pd.read_csv("test_X.csv")

In [8]:
dpps_features_test = test_x['ConstructedAASeq_cln'].apply(seq_to_dpps)
id = test_x.Id
X_test = pd.DataFrame(dpps_features_test.tolist())
X_test = scaler.transform(X_test)
y_pred = model.predict(X_test)
a = {"Id": id, "BrightnessClass": y_pred}
df = pd.DataFrame(a)
df.to_csv("Submit.csv", index = False)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifierCV

In [10]:
estimators = [
    ('lr', LogisticRegression(C = 0.1)),
    ('rg',RidgeClassifierCV(alphas=[1e-3, 1e-2, 1e-1, 1])),
    ("sgd", make_pipeline(StandardScaler(),SGDClassifier(max_iter=1000, tol=1e-3))),
    ('rf', RandomForestClassifier(n_estimators=1000, max_depth=10)),
]
clf = StackingClassifier(
    estimators=estimators, final_estimator=make_pipeline(StandardScaler(), LogisticRegression(C = 0.1))
)

In [11]:
clf.fit(X_train_scaled, y_train)

StackingClassifier(estimators=[('lr', LogisticRegression(C=0.1)),
                               ('rg',
                                RidgeClassifierCV(alphas=[0.001, 0.01, 0.1,
                                                          1])),
                               ('sgd',
                                Pipeline(steps=[('standardscaler',
                                                 StandardScaler()),
                                                ('sgdclassifier',
                                                 SGDClassifier())])),
                               ('rf',
                                RandomForestClassifier(max_depth=10,
                                                       n_estimators=1000))],
                   final_estimator=Pipeline(steps=[('standardscaler',
                                                    StandardScaler()),
                                                   ('logisticregression',
                                                    LogisticRegression(C=0.1))]))

In [12]:
y_preds = clf.predict(X_test_scaled)

In [13]:
precision_score(y_test, y_preds)

0.8519108280254777

In [14]:
f1_score(y_test, y_preds)

0.8685064935064936

In [15]:
accuracy_score(y_test, y_preds)

0.8955849178214631

In [16]:
dpps_features_test = test_x['ConstructedAASeq_cln'].apply(seq_to_dpps)
id = test_x.Id
X_test = pd.DataFrame(dpps_features_test.tolist())
X_test = scaler.transform(X_test)
y_pred = clf.predict(X_test)
a = {"Id": id, "BrightnessClass": y_pred}
df = pd.DataFrame(a)
df.to_csv("Submit.csv", index = False)